참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import matplotlib.patches as patch # 도형 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 08.007<br>ex08.007

모어의 원<br>Mohr's circle

p. 362

### 문제에서 주어진 변수<br>Given Parameters

2차원 응력<br>2D stress

In [ ]:
sigma_x_Pa, sigma_y_Pa, tau_xy_Pa = sy.symbols('sigma_{x}[Pa], sigma_{y}[Pa], tau_{xy}[Pa]', real=True)

s_d = {
    sigma_x_Pa: 40e6,
    sigma_y_Pa: 20e6,
    tau_xy_Pa: 16e6,
}


응력 상태 표시 준비<br>Prepare for stress status plot

In [ ]:
import os   # 운영체제 관련 기능 Operating Systems
import sys  # 시스템 관련 기능 Systems
# utils 폴더의 모듈을 import 할 수 있도록 준비
# add utils folder to sys.path to import
sys.path.append(os.path.abspath(os.path.join(os.pardir, 'utils')))
# 선도 관련 기능 diagrams
import draw_diagrams

응력상태를 표시함<br>Draw stress status

In [ ]:
draw_diagrams.draw_stress_2d(sigma_x_Pa.subs(s_d), 
                             sigma_y_Pa.subs(s_d), 
                             tau_xy_Pa.subs(s_d))
plt.show()

회전 각도<br>Rotation angle

In [ ]:
theta_deg = sy.symbols('theta[deg]', real=True)
theta_rad = sy.rad(theta_deg)

s_d[theta_deg] = 50


회전 각도를 라디안 단위로 표시<br>Rotation angle in radians

In [ ]:
theta_rad.subs(s_d)

### 모어의 원<br>Mohr's circle

p. 358 eq. 8.13 : 원의 중심의 $x$ 좌표<br>p. 358 eq. 8.13 : $x$ coordinate of the center of the circle

$$
\overline{\sigma}=\frac{\sigma_x + \sigma_y}{2}
$$

In [ ]:
sigma_bar_Pa = (sigma_x_Pa + sigma_y_Pa) / 2

In [ ]:
sigma_bar_Pa

MPa 단위값<br>In MPa unit

In [ ]:
sigma_bar_MPa = float(sigma_bar_Pa.subs(s_d)) * 1e-6

In [ ]:
sigma_bar_MPa

원의 중심을 표시함<br>Mark the center of the circle

In [ ]:
ax = plt.subplot(1, 2, 1)

ax.plot(sigma_bar_MPa, 0, 'k+')
ax.plot((0, 60), (0, 0), 'k+', alpha=0)

plt.axis('equal')
plt.grid(True)

plt.xlabel('$\\sigma$[MPa]')
plt.ylabel('$\\tau$[MPa]')

# 응력상태를 표시함 Indicate stress status
ax = plt.subplot(1, 2, 2)
draw_diagrams.draw_stress_2d(sigma_x_Pa.subs(s_d), 
                             sigma_y_Pa.subs(s_d), 
                             tau_xy_Pa.subs(s_d), ax=ax)

plt.show()
plt.clf()

p. 358 eq. 8.9 : 원의 반지름<br>p. 358 eq. 8.9 : Radius of the circle

$$
R=\sqrt { \left( \frac{\sigma_x - \sigma_y}{2} \right)^2 + \tau_{xy}^2 }
$$

In [ ]:
R_Pa = sy.sqrt(((sigma_x_Pa - sigma_y_Pa) / 2) ** 2 + tau_xy_Pa ** 2)

In [ ]:
R_Pa

MPa 단위값<br>In MPa unit

In [ ]:
R_MPa = float(R_Pa.subs(s_d)) * 1e-6

In [ ]:
R_MPa

원을 표시함<br>Draw circle

In [ ]:
fig, ax = plt.subplots()

ax.plot(sigma_bar_MPa, 0, 'k+')

circle = patch.Circle((sigma_bar_MPa, 0), R_MPa)
ax.add_patch(circle)

plt.axis('equal')
plt.grid(True)

plt.xlabel('$\\sigma$[MPa]')
plt.ylabel('$\\tau$[MPa]')

plt.show()
plt.clf()

주응력 방향<br>The principal direction

In [ ]:
b_Pa = sigma_x_Pa - sigma_bar_Pa

In [ ]:
h_Pa = - tau_xy_Pa

In [ ]:
theta_2_rad = float(sy.atan2(h_Pa, b_Pa).subs(s_d))

In [ ]:
theta_2_rad

In [ ]:
theta_2_deg = np.degrees(theta_2_rad)

In [ ]:
theta_2_deg

In [ ]:
delta_x_MPa, delta_y_MPa = R_MPa * np.cos(theta_2_rad), R_MPa * np.sin(theta_2_rad)

주응력 방향을 표시함<br>Indicate the principal direction

In [ ]:
ax = plt.subplot(1, 2, 1)

ax.plot(sigma_bar_MPa, 0, 'k+')

circle = patch.Circle((sigma_bar_MPa, 0), R_MPa)
ax.add_patch(circle)

ax.plot((sigma_bar_MPa + delta_x_MPa, sigma_bar_MPa - delta_x_MPa), (delta_y_MPa, -delta_y_MPa), 'k-')

plt.axis('equal')
plt.grid(True)

plt.xlabel('$\\sigma$[MPa]')
plt.ylabel('$\\tau$[MPa]')

ax2 = plt.subplot(1, 2, 2)
draw_diagrams.draw_stress_2d(sigma_bar_MPa + R_MPa, 
                             sigma_bar_MPa - R_MPa, 
                             0, ax=ax2, angle_deg = (-0.5) * theta_2_deg)

plt.show()
plt.clf()


또는 다음과 같이도 가능하다<br>Also available:

In [ ]:
ax = plt.subplot(1, 2, 1)
draw_diagrams.plot_mohr_circle(float(sigma_x_Pa.subs(s_d)), 
                               float(sigma_y_Pa.subs(s_d)), 
                               float(tau_xy_Pa.subs(s_d)), ax=ax)

ax2 = plt.subplot(1, 2, 2)
draw_diagrams.draw_stress_2d(sigma_bar_MPa + R_MPa, 
                             sigma_bar_MPa - R_MPa, 
                             0, ax=ax2, angle_deg = (-0.5) * theta_2_deg)

plt.show()
plt.clf()

### eq. 8.5

In [ ]:
theta_deg_array = np.arange(0, 360+1, 0.01)
theta_rad_array = np.deg2rad(theta_deg_array)

sx_Pa = 30e6
sy_Pa = 60e6
txy_Pa = 40e6

sx_sy_half_Pa = (sx_Pa + sy_Pa) * 0.5
sx_m_sy_half_Pa = (sx_Pa - sy_Pa) * 0.5

sx_Pa_array = sx_sy_half_Pa + sx_m_sy_half_Pa * np.cos(2*theta_rad_array) + txy_Pa * np.sin(2*theta_rad_array)
sy_Pa_array = sx_sy_half_Pa - sx_m_sy_half_Pa * np.cos(2*theta_rad_array) - txy_Pa * np.sin(2*theta_rad_array)
txy_Pa_array = - sx_m_sy_half_Pa * np.sin(2*theta_rad_array) + txy_Pa * np.cos(2*theta_rad_array)

sigma_xp_x = np.cos(theta_rad_array) * sx_Pa_array
sigma_xp_y = np.sin(theta_rad_array) * sx_Pa_array

sigma_yp_x = np.cos(theta_rad_array) * sy_Pa_array
sigma_yp_y = np.sin(theta_rad_array) * sy_Pa_array

tau_xpyp_x = np.cos(theta_rad_array) * txy_Pa_array
tau_xpyp_y = np.sin(theta_rad_array) * txy_Pa_array

plt.plot(sigma_xp_x, sigma_xp_y, label="$\\sigma_{x'}$")
plt.plot(sigma_yp_x, sigma_yp_y, label="$\\sigma_{y'}$")
plt.plot(tau_xpyp_x, tau_xpyp_y, label="$\\tau_{x'y'}$")

plt.plot([sx_Pa], [sy_Pa], '.')
plt.grid(True)
plt.axis('equal')
plt.legend(loc=0)

plt.show()